In [1]:
from imblearn.under_sampling import (RandomUnderSampler,
NearMiss,
InstanceHardnessThreshold,
CondensedNearestNeighbour,
EditedNearestNeighbours,
RepeatedEditedNearestNeighbours,
AllKNN,
NeighbourhoodCleaningRule,
OneSidedSelection,
TomekLinks)

In [2]:
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
data = pd.read_csv('train.csv')
ps = PorterStemmer()
def all_the_cooking(line):
    reviews = re.sub('[^a-zA-Z]' , ' ', line)
    reviews = reviews.lower()
    reviews = reviews.split()
    reviews = [ps.stem(word) for word in reviews if not word in set(stopwords.words('english'))]
    reviews = ' '.join(reviews)
    return(reviews)
for i,line in enumerate(data.tweet):
    data.tweet[i] = all_the_cooking(line)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [4]:
vect = TfidfVectorizer()
tfid_data_x = vect.fit_transform(data['tweet'])
train_y  = data['label']

In [5]:
tl = TomekLinks()
tl_xtrain_tfidf, tl_train_y = tl.fit_sample(tfid_data_x, train_y)

In [6]:
def train_model(train_x,train_y,test_x,test_y):
    from sklearn.naive_bayes import MultinomialNB
    model = MultinomialNB()
    model.fit(X=train_x , y = train_y)
    expected = test_y
    predicted=model.predict(test_x)
    from sklearn import metrics
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))
        

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_x, test_x, train_y, test_y = train_test_split(tl_xtrain_tfidf, tl_train_y)

In [11]:
train_model(train_x,train_y ,test_x,test_y)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      7410
           1       0.99      0.16      0.27       572

    accuracy                           0.94      7982
   macro avg       0.96      0.58      0.62      7982
weighted avg       0.94      0.94      0.92      7982

[[7409    1]
 [ 483   89]]
